In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
df = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/processed/cleaned_data.csv")
df.head()

,Total_Accidents,Total_Release_BBLS,Total_Recovered_BBLS,Avg_Prop_Damage,Avg_Env_Damage,Total_Fatalities,Total_Injuries,Operational_Rigs,Distillation_Capacity,Refinery_Inputs,%_Utilization_of_Refinery_Operable_Capacity,Date
0,34,7562.18,2590.44,16872.529412,236234.205882,0,0,433.0,17597.0,14065.0,79.9,2010-01-01
1,16,13791.04,29.95,78075.062500,15665.500000,0,0,446.0,17584.0,14267.0,81.1,2010-02-01
2,32,3663.14,2689.68,79171.843750,30998.281250,0,0,471.0,17584.0,14630.0,83.2,2010-03-01
3,32,3054.18,392.01,104276.687500,148810.875000,0,1,508.0,17589.0,15592.0,88.7,2010-04-01
4,30,5102.98,2841.72,949891.000000,75947.833333,0,0,541.0,17589.0,15510.0,88.2,2010-05-01


In [3]:
#Creating Lagged Features for previous month, 3 months ago, 6 months ago and 12 months ago

engineered_df = df.copy(deep=True)

for col in ["%_Utilization_of_Refinery_Operable_Capacity", "Refinery_Inputs", "Total_Accidents"]:
    for lag in [1,3,6,12]:
        engineered_df[f"{col}_lag{lag}"] = engineered_df[col].shift(lag)

In [4]:
#Creating rolling window features for 3, 6 and 12 months

for col in ["%_Utilization_of_Refinery_Operable_Capacity", "Total_Accidents", "Total_Release_BBLS", "Operational_Rigs"]:
    for w in [3,6,12]:
        engineered_df[f"{col}_roll{w}_mean"] = engineered_df[col].shift(1).rolling(w).mean()
        engineered_df[f"{col}_roll{w}_sum"] = engineered_df[col].shift(1).rolling(w).sum()


In [6]:
#Creating the new target column (utilization percentage + 1) meaning next month utilization percentage

engineered_df["target_utilization_pct"] = engineered_df["%_Utilization_of_Refinery_Operable_Capacity"].shift(-1)


In [8]:
#Dropping all NaNs

engineered_df = engineered_df.dropna().reset_index(drop=True)

In [12]:
#Creating our train, Evaluation and Holdout datasets

engineered_df = engineered_df.sort_values("Date").reset_index(drop=True)

#2010-2020
train_df = engineered_df[engineered_df["Date"] < "2021-01-01"]

#2020-2023
eval_df = engineered_df[(engineered_df["Date"] >= "2021-01-01") & (engineered_df["Date"] < "2024-01-01")]

#2024-2025
holdout_df = engineered_df[engineered_df["Date"] >= "2024-01-01"]



In [13]:
#Saving final datasets
import os

data_path = os.path.join("..", "data", "final_datasets")
os.makedirs(data_path, exist_ok=True)

datasets = {
    "train_df": train_df,
    "eval_df": eval_df,
    "holdout_df": holdout_df
}

for name, df in datasets.items():
    save_path = os.path.join(data_path, f"{name}.csv")
    df.to_csv(save_path, index=False)
    print(f"Saved {name}.csv to {save_path}")
print("All datasets saved")

Saved train_df.csv to ../data/final_datasets/train_df.csv
Saved eval_df.csv to ../data/final_datasets/eval_df.csv
Saved holdout_df.csv to ../data/final_datasets/holdout_df.csv
All datasets saved
